In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from icevision.models.multitask.ultralytics.yolov5 import *
from icevision.data.data_splitter import *
from icevision.visualize import *
from icevision.imports import *
import icedata.datasets.exdark_trimmed as exdark

In [3]:
IMG_SIZE=384
# data_dir = exdark.load_data()
data_dir = Path("/Users/rahulsomani/datasets/ExDark-Trimmed/")
parser = exdark.parser(data_dir)
train_ds, valid_ds = exdark.dataset(
    data_dir = data_dir,
    size = IMG_SIZE,
    data_splitter = RandomSplitter([0.8, 0.2])
)

  0%|          | 0/4626 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:136


  0%|          | 0/1850 [00:00<?, ?it/s]

AUTOFIX-SUCCESS - (record_id: 2178) - Clipping bbox ymax from 428 to image height 427 (Before: <BBox (xmin:3, ymin:245, xmax:205, ymax:428)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 512) - Clipping bbox xmax from 501 to image width 500 (Before: <BBox (xmin:91, ymin:5, xmax:501, ymax:204)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 1237) - Clipping bbox ymax from 536 to image height 528 (Before: <BBox (xmin:530, ymin:285, xmax:695, ymax:536)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 2063) - Clipping bbox ymax from 801 to image height 800 (Before: <BBox (xmin:399, ymin:657, xmax:758, ymax:801)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 323) - Clipping bbox xmax from 501 to image width 500 (Before: <BBox (xmin:441, ymin:95, xmax:501, ymax:132)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 199) - Clipping bbox ymax from

  0%|          | 0/463 [00:00<?, ?it/s]

AUTOFIX-SUCCESS - (record_id: 391) - Clipping bbox xmax from 641 to image width 640 (Before: <BBox (xmin:578, ymin:133, xmax:641, ymax:241)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 2098) - Clipping bbox xmax from 501 to image width 500 (Before: <BBox (xmin:439, ymin:202, xmax:501, ymax:270)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 2098) - Clipping bbox xmax from 501 to image width 500 (Before: <BBox (xmin:447, ymin:243, xmax:501, ymax:324)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 93) - Clipping bbox ymax from 376 to image height 375 (Before: <BBox (xmin:146, ymin:259, xmax:354, ymax:376)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 971) - Clipping bbox ymax from 376 to image height 375 (Before: <BBox (xmin:19, ymin:191, xmax:149, ymax:376)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 2182) - Clipping bbox xmax f

In [4]:
dl_train = train_dl(train_ds)
dl_valid = valid_dl(valid_ds)

In [5]:
hybrid_model = model(
    backbone=backbones.small(),
    num_detection_classes=len(parser.CLASS_MAPS['detection']),
    classifier_configs={
        name: ClassifierConfig(out_classes=len(cm))
        for name, cm in parser.CLASS_MAPS.items() if not name=="detection"
    },
    img_size=IMG_SIZE,
)


WARNING  - Incompatible `num_fpn_features=512` detected in task 'lighting'. Replacing with the correct dimensions: 896 | icevision.models.multitask.ultralytics.yolov5.yolo_hybrid:build_classifier_heads:180
WARNING  - Incompatible `num_fpn_features=512` detected in task 'location'. Replacing with the correct dimensions: 896 | icevision.models.multitask.ultralytics.yolov5.yolo_hybrid:build_classifier_heads:180
SUCCESS  - Built classifier heads successfully | icevision.models.multitask.ultralytics.yolov5.yolo_hybrid:build_classifier_heads:188
INFO     - Overriding model.yaml nc=80 with nc=12 | icevision.models.multitask.ultralytics.yolov5.yolo_hybrid:__init__:119
SUCCESS  - Built *yolov5s* model successfully | icevision.models.multitask.ultralytics.yolov5.yolo_hybrid:__init__:150


In [6]:
from torch import optim
import pytorch_lightning as pl

class LightModel(lightning.HybridYOLOV5LightningAdapter):
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=3e-4)

pl_model = LightModel(
    model=hybrid_model,
    metrics=None,
)

In [9]:
trainer = pl.Trainer(max_epochs=3, gpus=[0])
trainer = pl.Trainer(
    max_epochs=3,
    # limit_train_batches=10,
    # limit_val_batches=2,
)
trainer

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(pl_model, dl_train, dl_valid)